In [7]:
import time
import random
import glob

import json
import numpy as np

from shapely.geometry import shape, Point
from shapely.ops import unary_union
import geopandas as gpd
import pandas as pd

import fiona

from alive_progress import alive_bar
import geopy.distance

In [2]:
def load_shapefile(file_path):
    with fiona.open(file_path, 'r') as shp:
        # read the features and handle broken geometries
        features = []
        with alive_bar(len(shp), title="Loading shapefile...", force_tty=True) as bar:
            for feature in shp:
                try:
                    # create a valid feature by fixing the geometry
                    valid_feature = {'geometry': shape(feature['geometry']).buffer(
                        0), 'properties': feature['properties']}
                    features.append(valid_feature)
                except Exception as e:
                    print("Error fixing geometry, skipping feature")
                bar()

    gdf = gpd.GeoDataFrame.from_features(features)
    gdf.crs = shp.crs
    # only keep polygons
    gdf = gdf[gdf.geometry.type != 'Point']

    return gdf

In [28]:
def standardize_causes(df): 

    # remap usfs cause codes to their corresponding causes
    usfs_cause_map = {
        '1': 'lightning',
        '2': 'equipment',
        '3': 'smoking',
        '4': 'campfire',
        '5': 'debris burning',
        '6': 'railroad',
        '7': 'arson',
        '8': 'children',
        '9': 'miscellaneous',
    }

    for num, cause in usfs_cause_map.items():
        df.loc[df['CAUSE_DETAIL_2'] == num, 'CAUSE_DETAIL_2'] = cause

    # remap layer 2 causes to a standardized set of cases
    layer_2_cause_map = {
        None: None,
        'campfire': 'camping',
        'equipment': 'equipment and vehicle use',
        'children': 'misuse of fire by a minor',
        '5-debris burning': 'debris and open burning',
        'debris/open burning': 'debris and open burning',
        'debris burning': 'debris and open burning',
        'railroad': 'railroad operations and maintenance',
        'firearms and explosives use': 'firearms and weapons',
        'firearms/weapons': 'firearms and weapons',
        'power generation/transmission/distribution': 'utilities',
        'incindiary':'incendiary',
        '7-arson': 'arson',
        'undetermined': None,
        'miscellaneous': None,
        'undetermined (remarks required)': None,
        'cause and origin not identified': None,
        'investigated but undetermined': None,
        'investigated but und': None,
        'cause not identified': None,
        'undetermined (remar*': None,
        '9 - miscellaneous': None,
        '10': None,
        '14': None,
        '0': None
    }

    for og_cause, cause in layer_2_cause_map.items():
        df.loc[df['CAUSE_DETAIL_2'] == og_cause, 'CAUSE_DETAIL_2'] = cause
        
    # existing layer 2 causes common in both datasets 
    # 'incendiary'
    # 'lightning'
    # 'camping'
    # 'recreation and ceremony'
    # 'other human cause'
    # 'other natural cause'
    # 'arson'
    # 'coal seam'
    # 'smoking'
    # 'utilities'

    # generate layer 1 causes from layer 2 causes
    layer_1_cause_map = {
        'human': 'human',
        'natural': 'natural',
        None: None,
        'equipment and vehicle use': 'human',
        'misuse of fire by a minor': 'human',
        'debris and open burning': 'human',
        'railroad operations and maintenance': 'human',
        'firearms and weapons': 'human',
        'incendiary': 'human',
        'camping': 'human',
        'recreation and ceremony': 'human',
        'arson': 'human',
        'smoking': 'human',
        'utilities':'human',
        'other human cause': 'human',
        'coal seam': 'human',
        'lightning': 'natural',
        'other natural cause': 'natural',
        'volcanic': 'natural',
        'undetermined': None
    }

    def create_layer_1_value(row):
        if pd.isnull(row['CAUSE_DETAIL_1']):
            try:    
                return layer_1_cause_map[row['CAUSE_DETAIL_2']]
            except KeyError as e:
                return None
        return row['CAUSE_DETAIL_1']

    df['CAUSE_DETAIL_1'] = df.apply(create_layer_1_value, axis=1)

    return df

In [8]:
# Load USFS fire dataset.
usfs_path = "data/USFSPerimeters/*.shp"
if len(glob.glob(usfs_path)) == 0:
    raise Exception("No USFS shapefile found at path {}".format(usfs_path))
else:
    usfs_df = load_shapefile(
        glob.glob(usfs_path)[0])


usfs_df.head()

on 42021: Error fixing geometry, skipping feature                               |█████████                               | ▆█▆ 12600/55781 |█████████████████████████████▋          | ▄▆█ 41315/55781 
on 42022: Error fixing geometry, skipping feature                               
on 42023: Error fixing geometry, skipping feature                               
on 42353: Error fixing geometry, skipping feature                               
on 42842: Error fixing geometry, skipping feature                               
on 42843: Error fixing geometry, skipping feature                               
on 42844: Error fixing geometry, skipping feature                               
on 43344: Error fixing geometry, skipping feature                               
on 50170: Error fixing geometry, skipping feature                               
on 50317: Error fixing geometry, skipping feature                               
on 50328: Error fixing geometry, skipping feature                       

,geometry,OBJECTID,GLOBALID,FIREOCCURI,CN,REVDATE,FIRENAME,COMPLEXNAM,FIREYEAR,UNIQFIREID,...,PROTECTION,UNITIDPROT,FEATURECAT,SHAPEAREA,SHAPELEN,DBSOURCEID,DBSOURCEDA,SECURITYID,FIRERPTQC,ACCURACY
0,"POLYGON ((-115.34676 46.09877, -115.34568 46.0...",7079850,{59A54B0D-D66C-459C-A930-E3CAA9E55804},None,1166368,None,Glover,None,2003.0,2003-IDNCF-000141,...,None,None,Final Wildfire Perimeter,0.000006,0.011895,01,2023-07-22,0117,None,NaN
1,"POLYGON ((-114.89249 46.05801, -114.89286 46.0...",7079851,{AA2E13E6-8BE6-4C92-8A57-FA66E496F605},None,257682,None,Patrol Point,None,1992.0,1992-IDNCF-000214,...,None,None,Final Wildfire Perimeter,0.000007,0.009547,01,2023-07-22,0117,None,NaN
2,"POLYGON ((-115.11075 46.21926, -115.10959 46.2...",7079852,{C438DD5F-57D2-46B8-BE5B-E21506DD0C92},None,288986,None,Blacktail,None,1994.0,1994-IDNCF-000285,...,None,None,Final Wildfire Perimeter,0.000016,0.016867,01,2023-07-22,0117,None,NaN
3,"POLYGON ((-112.91254 48.12448, -112.91245 48.1...",7079853,{14DE3220-543D-40C6-B8D1-D819B375D9DF},None,None,None,None,None,1910.0,None,...,None,None,Final Wildfire Perimeter,0.005640,0.352226,01,2023-07-22,0115,None,NaN
4,"POLYGON ((-116.24875 45.44812, -116.24856 45.4...",7079854,{66C8A326-F477-4BA3-BBE7-2492661A8B68},None,1392987,None,Lightning,None,2006.0,2006-IDNCF-000102,...,None,None,Final Wildfire Perimeter,0.000909,0.143848,01,2023-07-22,0117,None,NaN


In [9]:
# Select only the columns of interest.

# https://data.fs.usda.gov/geodata/edw/edw_resources/meta/S_USA.FinalFirePerimeter.xml
usfs_remap = {
    'FIRENAME': 'NAME',
    'FIREYEAR': 'YEAR',
    'OWNERAGENC': 'AGENCY',
    'STATCAUSE': 'CAUSE_DETAIL_2',
    'DISCOVERYD': 'STARTDATE',
    'geometry': 'geometry',
}
usfs_df_sel = usfs_df.rename(columns=usfs_remap)
usfs_df_sel = usfs_df_sel[list(usfs_remap.values())]
usfs_df_sel.head()

,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry
0,Glover,2003.0,USFS,Lightning,None,"POLYGON ((-115.34676 46.09877, -115.34568 46.0..."
1,Patrol Point,1992.0,USFS,Lightning,None,"POLYGON ((-114.89249 46.05801, -114.89286 46.0..."
2,Blacktail,1994.0,USFS,Lightning,None,"POLYGON ((-115.11075 46.21926, -115.10959 46.2..."
3,None,1910.0,None,Undetermined,None,"POLYGON ((-112.91254 48.12448, -112.91245 48.1..."
4,Lightning,2006.0,USFS,Lightning,None,"POLYGON ((-116.24875 45.44812, -116.24856 45.4..."


In [29]:
# Standardize the USFS dataset.

# no endtime in this dataset, but add the column anyway
usfs_df_st = usfs_df_sel.copy()
usfs_df_st['ENDDATE'] = None
usfs_df_st['SOURCE'] = 'USFS'


# drop rows where year is null
usfs_df_st = usfs_df_st.dropna(subset=['YEAR'])

# convert year to datetime
usfs_df_st['YEAR'] = usfs_df_st['YEAR'].astype(int).astype(str)

# clean up name values
usfs_df_st['NAME'] = usfs_df_st['NAME'].str.lower()
usfs_df_st['NAME'].fillna('unnamed fire', inplace=True)
usfs_df_st['NAME'] = usfs_df_st['NAME'].replace('noname', 'unnamed fire')
usfs_df_st['NAME'] = usfs_df_st['NAME'].replace('unknown', 'unnamed fire')
usfs_df_st['NAME'] = usfs_df_st['NAME'].replace('missing', 'unnamed fire')
usfs_df_st['NAME'] = usfs_df_st['NAME'].replace('n/a', 'unnamed fire')

# clean up cause values 
usfs_df_st['CAUSE_DETAIL_2'] = usfs_df_st['CAUSE_DETAIL_2'].str.lower()
usfs_df_st['CAUSE_DETAIL_1'] = None
usfs_df_st['CAUSE_DETAIL_3'] = None

usfs_df_st = standardize_causes(usfs_df_st)
    
usfs_df_st.head()

,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry,ENDDATE,SOURCE,CAUSE_DETAIL_1,CAUSE_DETAIL_3
0,glover,2003,USFS,lightning,None,"POLYGON ((-115.34676 46.09877, -115.34568 46.0...",None,USFS,natural,None
1,patrol point,1992,USFS,lightning,None,"POLYGON ((-114.89249 46.05801, -114.89286 46.0...",None,USFS,natural,None
2,blacktail,1994,USFS,lightning,None,"POLYGON ((-115.11075 46.21926, -115.10959 46.2...",None,USFS,natural,None
3,unnamed fire,1910,None,None,None,"POLYGON ((-112.91254 48.12448, -112.91245 48.1...",None,USFS,None,None
4,lightning,2006,USFS,lightning,None,"POLYGON ((-116.24875 45.44812, -116.24856 45.4...",None,USFS,natural,None


In [7]:
# Load BLM fire dataset.
blm_df = load_shapefile(
    'data/BLMPerimeters/fper_poly.shp')

blm_df.head()

on 23393: Error fixing geometry, skipping feature                               
Loading shapefile... |████████████████████████████████████████| 24945/24945 [100


,geometry,OBJECTID,POO_JRSDCT,POO_PRTCT_,RPT_UNIT_I,LOCAL_INCD,FIRE_CODE_,INCDNT_NM,FIRE_DSCVR,FIRE_DSC_1,...,MODIFY_DAT,MODIFY_BY,created_us,created_da,last_edite,last_edi_1,Original_G,UNQE_FIRE_,SHAPE_Leng,SHAPE_Area
0,"POLYGON ((-113.15345 42.16329, -113.15317 42.1...",86126,USIDSTF,USIDSTF,None,433.0,G45Y,8 MILE,2012.0,2012-08-07,...,9999-09-09,UNK,None,None,None,None,{4D31CFCB-9A13-4B22-B786-613DF36CE3E5},2012--000433,0.047309,0.000093
1,"POLYGON ((-113.17363 42.10555, -113.17294 42.1...",86129,USIDSTF,USIDSTF,None,432.0,G43T,DEER HOLLOW,2012.0,2012-08-07,...,9999-09-09,UNK,None,None,None,None,{68F89D03-2991-4E80-AAC9-76CA2094AE1C},2012--000432,0.382020,0.002396
2,"POLYGON ((-114.52680 43.59520, -114.52690 43.5...",86135,USIDSTF,USIDSTF,None,474.0,G6CZ,ENCLOSURE,2012.0,2012-08-16,...,9999-09-09,UNK,None,None,None,None,{5FCCA90C-B856-4EA7-8775-D9920E789254},2012--000474,0.043329,0.000105
3,"POLYGON ((-113.04998 42.05654, -113.04998 42.0...",86161,USIDSTF,USIDSTF,None,267.0,G1DX,BLACK MINE,2012.0,2012-07-07,...,9999-09-09,UNK,None,None,None,None,{3C9EB863-371A-4DD2-9E4F-564BF9BFC113},2012--000267,0.050783,0.000029
4,"POLYGON ((-112.92440 42.29591, -112.92440 42.2...",86167,USIDSTF,USIDSTF,None,633.0,G9PK,PINE SPRING,2012.0,2012-10-11,...,9999-09-09,UNK,None,None,None,None,{AC468B25-F3B0-4C0A-A881-51524862FE19},2012--000633,0.061491,0.000061


In [8]:
# Select only the columns of interest.

# https://gbp-blm-egis.hub.arcgis.com/datasets/BLM-EGIS::blm-natl-fire-perimeters-polygon/about
blm_remap = {
    'INCDNT_NM': 'NAME',
    'FIRE_DSCVR': 'YEAR',
    'FIRE_CAUSE': 'CAUSE_DETAIL_1',
    'FIRE_DSC_1': 'STARTDATE',
    'FIRE_CNTRL': 'ENDDATE',
    'geometry': 'geometry',
}
blm_df_sel = blm_df.rename(columns=blm_remap)
blm_df_sel = blm_df_sel[list(blm_remap.values())]
blm_df_sel.head()

,NAME,YEAR,CAUSE_DETAIL_1,STARTDATE,ENDDATE,geometry
0,8 MILE,2012.0,Natural,2012-08-07,2012-12-31,"POLYGON ((-113.15345 42.16329, -113.15317 42.1..."
1,DEER HOLLOW,2012.0,Natural,2012-08-07,2012-12-31,"POLYGON ((-113.17363 42.10555, -113.17294 42.1..."
2,ENCLOSURE,2012.0,Human,2012-08-16,2012-12-31,"POLYGON ((-114.52680 43.59520, -114.52690 43.5..."
3,BLACK MINE,2012.0,Natural,2012-07-07,2012-12-31,"POLYGON ((-113.04998 42.05654, -113.04998 42.0..."
4,PINE SPRING,2012.0,Human,2012-10-11,2012-12-31,"POLYGON ((-112.92440 42.29591, -112.92440 42.2..."


In [9]:
# Standardize the BLM dataset.

blm_df_st = blm_df_sel.copy()
blm_df_st['SOURCE'] = 'BLM'

# drop rows where year is null
blm_df_st = blm_df_st.dropna(subset=['YEAR'])

# convert year to datetime
blm_df_st['YEAR'] = blm_df_st['YEAR'].astype(int).astype(str)

# clean up name values
blm_df_st['NAME'] = blm_df_st['NAME'].str.lower()
blm_df_st['NAME'].fillna('unnamed fire', inplace=True)

# standardize causes
blm_df_st['CAUSE_DETAIL_1'] = blm_df_st['CAUSE_DETAIL_1'].str.lower()
blm_df_st['CAUSE_DETAIL_1'] = blm_df_st['CAUSE_DETAIL_1'].replace('uk', None)
blm_df_st['CAUSE_DETAIL_1'] = blm_df_st['CAUSE_DETAIL_1'].replace('unknown', None)
blm_df_st['CAUSE_DETAIL_1'].fillna('undetermined', inplace=True)
blm_df_st['CAUSE_DETAIL_2'] = None
blm_df_st['CAUSE_DETAIL_3'] = None

blm_df_st['ENDDATE'] = blm_df_st['ENDDATE'].replace('9999-09-09', None)
    
blm_df_st.head()

,NAME,YEAR,CAUSE_DETAIL_1,STARTDATE,ENDDATE,geometry,SOURCE,CAUSE_DETAIL_2,CAUSE_DETAIL_3
0,8 mile,2012,natural,2012-08-07,2012-12-31,"POLYGON ((-113.15345 42.16329, -113.15317 42.1...",BLM,None,None
1,deer hollow,2012,natural,2012-08-07,2012-12-31,"POLYGON ((-113.17363 42.10555, -113.17294 42.1...",BLM,None,None
2,enclosure,2012,human,2012-08-16,2012-12-31,"POLYGON ((-114.52680 43.59520, -114.52690 43.5...",BLM,None,None
3,black mine,2012,natural,2012-07-07,2012-12-31,"POLYGON ((-113.04998 42.05654, -113.04998 42.0...",BLM,None,None
4,pine spring,2012,human,2012-10-11,2012-12-31,"POLYGON ((-112.92440 42.29591, -112.92440 42.2...",BLM,None,None


In [10]:
# Load NIFC fire dataset.
nifc_df = load_shapefile(
    'data/NIFCPerimeters/Perimeters.shp')

nifc_df.head()

on 2774: Error fixing geometry, skipping feature                                
on 14456: Error fixing geometry, skipping feature                               |█████████████████████████████████▋      | ▃▅▇ 14272/16948 
on 14469: Error fixing geometry, skipping feature                               
on 14483: Error fixing geometry, skipping feature                               
on 14496: Error fixing geometry, skipping feature                               
on 14596: Error fixing geometry, skipping feature                               
on 14654: Error fixing geometry, skipping feature                               
on 14659: Error fixing geometry, skipping feature                               
Loading shapefile... |████████████████████████████████████████| 16948/16948 [100


,geometry,OBJECTID,poly_Sourc,poly_Incid,poly_Featu,poly_MapMe,poly_GISAc,poly_Creat,poly_DateC,poly_Polyg,...,attr_Cre_1,attr_Mod_1,attr_Sou_1,attr_IsCpx,attr_CpxNa,attr_CpxID,attr_Sou_2,GlobalID,SHAPE_Leng,SHAPE_Area
0,"POLYGON ((-122.51244 46.34552, -122.51243 46.3...",12,NaN,PULLEN CREEK,Wildfire Daily Fire Perimeter,GPS-Walked,19.500000,2022-11-19,2022-12-16,2022-11-19,...,2022-11-19,2022-12-28,IRWIN,0.0,None,None,{5D73EEC4-8D85-4843-9D9C-141731517538},{E78D410B-99D1-495E-867E-70D9849988F5},0.017515,-0.000009
1,"MULTIPOLYGON (((-121.49952 47.81673, -121.4992...",17,NaN,Bolt Creek,Wildfire Daily Fire Perimeter,Mixed Methods,14820.544258,2022-10-20,2022-10-20,2022-10-20,...,2022-09-10,2022-12-22,IRWIN,0.0,None,None,{D593321D-9118-4046-ABEE-E1C40B69035D},{2CAE4FC6-27E0-43AE-8F7F-D1BBDED23C32},1.601665,-0.007195
2,"MULTIPOLYGON (((-122.32396 43.89499, -122.3239...",21,NaN,Cedar Creek,Wildfire Daily Fire Perimeter,IR Image Interpretation,127310.968593,2022-10-08,2022-10-28,2022-10-19,...,2022-08-02,2023-06-12,IRWIN,0.0,None,None,{21BB61C1-C479-4721-8295-9120974092D2},{F00C42C6-8F11-4FEE-9D50-F293D9F51EA0},4.850371,-0.057600
3,"POLYGON ((-111.55901 33.65397, -111.55814 33.6...",22,52195.0,Musket,Wildfire Daily Fire Perimeter,Hand Sketch,0.360000,2022-12-19,2022-12-20,2022-12-17,...,2022-12-19,2022-12-19,IRWIN,0.0,None,None,{3DA6FD6E-78F0-4334-8F8A-4237117BD8E1},{1CA86506-881C-4CBF-A6E1-77AD6AD6C59B},0.006301,-0.000002
4,"POLYGON ((-96.85159 30.52503, -96.84789 30.526...",23,NaN,Saltillo,Wildfire Daily Fire Perimeter,Mixed Methods,120.897078,2022-08-03,2022-08-03,2022-08-03,...,2022-08-02,2022-12-22,IRWIN,0.0,None,None,{375B387E-B68C-4941-AFBA-A808799D6EA4},{FB01DA94-C49E-42CE-923D-DDCF7D26B92F},0.038010,-0.000046


In [11]:
nifc_df[nifc_df["poly_Incid"] == "Holiday Farm"]

,geometry,OBJECTID,poly_Sourc,poly_Incid,poly_Featu,poly_MapMe,poly_GISAc,poly_Creat,poly_DateC,poly_Polyg,...,attr_Cre_1,attr_Mod_1,attr_Sou_1,attr_IsCpx,attr_CpxNa,attr_CpxID,attr_Sou_2,GlobalID,SHAPE_Leng,SHAPE_Area
13856,"MULTIPOLYGON (((-122.23629 44.20864, -122.2362...",14117,2274.0,Holiday Farm,Wildfire Daily Fire Perimeter,Mixed Methods,173393.0,2021-05-04,2021-05-04,2020-10-07,...,2020-09-08,2021-09-27,IRWIN,0.0,None,None,{D0D17250-64E5-4961-A1DC-42986547C3FF},{0C04E993-BDA5-4347-BAA1-0D2C5142DBA2},4.744982,-0.078935


In [12]:
# Select only the columns of interest.

# https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-interagency-fire-perimeters/about
nifc_remap = {
    'poly_Incid': 'NAME',
    # XXX: 'YEAR', # will need to calculate this from STARTDATE
    'attr_FireC': 'CAUSE_DETAIL_1',
    'attr_Fir_4': 'CAUSE_DETAIL_2',
    'attr_Fir_5': 'CAUSE_DETAIL_3',
    'attr_POOLa': 'AGENCY',
    'attr_Fir_7': 'STARTDATE', 
    'attr_Conta': 'ENDDATE',
    'geometry': 'geometry',
}
nifc_df_sel = nifc_df.rename(columns=nifc_remap)
nifc_df_sel = nifc_df_sel[list(nifc_remap.values())]
nifc_df_sel.head()

,NAME,CAUSE_DETAIL_1,CAUSE_DETAIL_2,CAUSE_DETAIL_3,AGENCY,STARTDATE,ENDDATE,geometry
0,PULLEN CREEK,Undetermined,None,None,Private,2022-11-19,2022-11-20,"POLYGON ((-122.51244 46.34552, -122.51243 46.3..."
1,Bolt Creek,Human,None,None,USFS,2022-09-10,None,"MULTIPOLYGON (((-121.49952 47.81673, -121.4992..."
2,Cedar Creek,Natural,None,None,USFS,2022-08-01,None,"MULTIPOLYGON (((-122.32396 43.89499, -122.3239..."
3,Musket,Human,None,None,USFS,2022-12-17,2022-12-17,"POLYGON ((-111.55901 33.65397, -111.55814 33.6..."
4,Saltillo,Human,Power generation/transmission/distribution,Electrical transmission/distribution systems,Private,2022-08-02,2022-08-05,"POLYGON ((-96.85159 30.52503, -96.84789 30.526..."


In [13]:
# Standardize the NIFC dataset.

nifc_df_st = nifc_df_sel.copy()

# compute year from STARTDATE
nifc_df_st['YEAR'] = [e[:4] for e in nifc_df_st['STARTDATE']]

nifc_df_st['SOURCE'] = 'NIFC'

# clean up name values
nifc_df_st['NAME'] = nifc_df_st['NAME'].str.lower()
nifc_df_st['NAME'].fillna('unnamed fire', inplace=True)
nifc_df_st['NAME'] = nifc_df_st['NAME'].replace('noname', 'unnamed fire')
nifc_df_st['NAME'] = nifc_df_st['NAME'].replace('unknown', 'unnamed fire')
nifc_df_st['NAME'] = nifc_df_st['NAME'].replace('missing', 'unnamed fire')
nifc_df_st['NAME'] = nifc_df_st['NAME'].replace('n/a', 'unnamed fire')

# clean up cause values 
nifc_df_st['CAUSE_DETAIL_1'] = nifc_df_st['CAUSE_DETAIL_1'].str.lower()
nifc_df_st['CAUSE_DETAIL_2'] = nifc_df_st['CAUSE_DETAIL_2'].str.lower()
nifc_df_st['CAUSE_DETAIL_3'] = nifc_df_st['CAUSE_DETAIL_3'].str.lower()

nifc_df_st = standardize_causes(nifc_df_st)
    
nifc_df_st.head()

,NAME,CAUSE_DETAIL_1,CAUSE_DETAIL_2,CAUSE_DETAIL_3,AGENCY,STARTDATE,ENDDATE,geometry,YEAR,SOURCE
0,pullen creek,undetermined,None,None,Private,2022-11-19,2022-11-20,"POLYGON ((-122.51244 46.34552, -122.51243 46.3...",2022,NIFC
1,bolt creek,human,None,None,USFS,2022-09-10,None,"MULTIPOLYGON (((-121.49952 47.81673, -121.4992...",2022,NIFC
2,cedar creek,natural,None,None,USFS,2022-08-01,None,"MULTIPOLYGON (((-122.32396 43.89499, -122.3239...",2022,NIFC
3,musket,human,None,None,USFS,2022-12-17,2022-12-17,"POLYGON ((-111.55901 33.65397, -111.55814 33.6...",2022,NIFC
4,saltillo,human,utilities,electrical transmission/distribution systems,Private,2022-08-02,2022-08-05,"POLYGON ((-96.85159 30.52503, -96.84789 30.526...",2022,NIFC


In [14]:
all_fires_df = pd.concat([usfs_df_st, blm_df_st, nifc_df_st])

# remove old fires
all_fires_df = all_fires_df[all_fires_df['STARTDATE'] >= '1900-01-01']

all_fires_df.head()

,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry,ENDDATE,SOURCE,CAUSE_DETAIL_1,CAUSE_DETAIL_3
27,rain dance,1991,USFS,camping,1991-10-10,"POLYGON ((-115.12240 46.27461, -115.12312 46.2...",None,USFS,human,None
40,balinger,1987,USFS,camping,1987-10-14,"POLYGON ((-115.15423 46.10623, -115.15273 46.1...",None,USFS,human,None
52,goat,2012,USFS,lightning,2012-01-01,"POLYGON ((-114.86655 45.98484, -114.86660 45.9...",None,USFS,natural,None
81,sweeney,2012,BLM,lightning,2012-08-01,"POLYGON ((-106.17360 46.13986, -106.17376 46.1...",None,USFS,natural,None
87,lynch,2012,MT,lightning,2012-07-10,"POLYGON ((-107.17635 47.05784, -107.17634 47.0...",None,USFS,natural,None


In [15]:
# Calculate the centroids of each fire polygon.
all_fires_df['CENTROID'] = all_fires_df.geometry.centroid.apply(Point)
all_fires_df.head()

,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry,ENDDATE,SOURCE,CAUSE_DETAIL_1,CAUSE_DETAIL_3,CENTROID
27,rain dance,1991,USFS,camping,1991-10-10,"POLYGON ((-115.12240 46.27461, -115.12312 46.2...",None,USFS,human,None,POINT (-115.12390 46.28263)
40,balinger,1987,USFS,camping,1987-10-14,"POLYGON ((-115.15423 46.10623, -115.15273 46.1...",None,USFS,human,None,POINT (-115.15293 46.10515)
52,goat,2012,USFS,lightning,2012-01-01,"POLYGON ((-114.86655 45.98484, -114.86660 45.9...",None,USFS,natural,None,POINT (-114.86665 45.99264)
81,sweeney,2012,BLM,lightning,2012-08-01,"POLYGON ((-106.17360 46.13986, -106.17376 46.1...",None,USFS,natural,None,POINT (-106.20312 46.15866)
87,lynch,2012,MT,lightning,2012-07-10,"POLYGON ((-107.17635 47.05784, -107.17634 47.0...",None,USFS,natural,None,POINT (-107.17344 47.05855)


In [16]:
# Calculate the area of each fire polygon (m^2).

# https://gis.stackexchange.com/questions/413349/calculating-area-of-lat-lon-polygons-without-transformation-using-geopandas
from shapely.geometry.polygon import orient
def gpd_geographic_area(geodf):
    if not (geodf.crs and geodf.crs.is_geographic):
        raise TypeError('geodataframe should have geographic coordinate system')
        
    geod = geodf.crs.get_geod()
    def area_calc(geom):
        if geom.geom_type not in ['MultiPolygon','Polygon']:
            return np.nan
        
        # For MultiPolygon do each separately
        if geom.geom_type=='MultiPolygon':
            return np.sum([area_calc(p) for p in geom.geoms])

        # orient to ensure a counter-clockwise traversal. 
        # See https://pyproj4.github.io/pyproj/stable/api/geod.html
        # geometry_area_perimeter returns (area, perimeter)
        return geod.geometry_area_perimeter(orient(geom, 1))[0]
    
    return geodf.geometry.apply(area_calc)

all_fires_df.set_crs("EPSG:4326", inplace=True)
all_fires_df['AREA'] = gpd_geographic_area(all_fires_df)
all_fires_df.head()

,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry,ENDDATE,SOURCE,CAUSE_DETAIL_1,CAUSE_DETAIL_3,CENTROID,AREA
27,rain dance,1991,USFS,camping,1991-10-10,"POLYGON ((-115.12240 46.27461, -115.12312 46.2...",None,USFS,human,None,POINT (-115.12390 46.28263),2.728983e+06
40,balinger,1987,USFS,camping,1987-10-14,"POLYGON ((-115.15423 46.10623, -115.15273 46.1...",None,USFS,human,None,POINT (-115.15293 46.10515),6.191073e+04
52,goat,2012,USFS,lightning,2012-01-01,"POLYGON ((-114.86655 45.98484, -114.86660 45.9...",None,USFS,natural,None,POINT (-114.86665 45.99264),9.043758e+06
81,sweeney,2012,BLM,lightning,2012-08-01,"POLYGON ((-106.17360 46.13986, -106.17376 46.1...",None,USFS,natural,None,POINT (-106.20312 46.15866),5.117821e+07
87,lynch,2012,MT,lightning,2012-07-10,"POLYGON ((-107.17635 47.05784, -107.17634 47.0...",None,USFS,natural,None,POINT (-107.17344 47.05855),2.070669e+05


In [17]:
# Create a df ready to export as a shapefile.

out_df = all_fires_df.copy()

# export to shapefile does not handle GeometryDtype
out_df = out_df.drop(columns=['CENTROID'])

# set projection so can properly save geometry
out_df.set_crs("EPSG:4326", inplace=True)

print(out_df.head())

# Convert dataframe to shapefile to analyze in QGIS.
out_df.to_file("data/all_fire_groups.shp", driver='ESRI Shapefile')

          NAME  YEAR AGENCY CAUSE_DETAIL_2   STARTDATE  \
27  rain dance  1991   USFS        camping  1991-10-10   
40    balinger  1987   USFS        camping  1987-10-14   
52        goat  2012   USFS      lightning  2012-01-01   
81     sweeney  2012    BLM      lightning  2012-08-01   
87       lynch  2012     MT      lightning  2012-07-10   

                                             geometry ENDDATE SOURCE  \
27  POLYGON ((-115.12240 46.27461, -115.12312 46.2...    None   USFS   
40  POLYGON ((-115.15423 46.10623, -115.15273 46.1...    None   USFS   
52  POLYGON ((-114.86655 45.98484, -114.86660 45.9...    None   USFS   
81  POLYGON ((-106.17360 46.13986, -106.17376 46.1...    None   USFS   
87  POLYGON ((-107.17635 47.05784, -107.17634 47.0...    None   USFS   

   CAUSE_DETAIL_1 CAUSE_DETAIL_3          AREA  
27          human           None  2.728983e+06  
40          human           None  6.191073e+04  
52        natural           None  9.043758e+06  
81        natural     

/tmp/ipykernel_30808/4126802563.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  out_df.to_file("data/all_fire_groups.shp", driver='ESRI Shapefile')


In [18]:
# Find groups of rows that likely represent the same fire.

T = 500 # threshold = 500 meters

def same_heuristic(r1, r2):
    c1 = r1['CENTROID']
    c2 = r2['CENTROID']
    dist = geopy.distance.geodesic((c1.y, c1.x), (c2.y, c2.x))
    return dist.meters < T

def find_fire_groups(df):
    groups = df.groupby(['NAME', 'YEAR'])
    
    with alive_bar(groups.ngroups, title="Finding distinct fires...", force_tty=True) as bar:
        fire_groups_list = []
        for name, group in groups: 
            n = len(group.index)
            colors = np.arange(n)
        
            # color rows by first row that is transitively close to it
            for i in range(n):
                for j in range(i+1, n):
                    if same_heuristic(group.iloc[i], group.iloc[j]): colors[j] = colors[i]
        
            group['COLOR'] = colors
            cgroups = group.groupby(['COLOR'])
            fire_groups_list += [cgroup for _, cgroup in cgroups]
    
            bar()
    
    fire_df = pd.concat(fire_groups_list)
    fire_groups = fire_df.groupby(['NAME', 'YEAR', 'COLOR'])
    return fire_groups

fire_groups = find_fire_groups(all_fires_df)
fire_groups.ngroups

Finding distinct fires... |████████████████████████████████████████| 62419/62419|██████████████████▋                     | ▂▄▆ 29101/6|█████████████████████████████████████▍  | ▁▃▅ 58248/6|█████████████████████████████████████▍  | ▅▃▁ 58256/6|██████████████████████████████████████▌ | ▄▆█ 60109/6


72519

In [19]:
class GroupError(Exception):
    pass
    
def analysis(group):
    # Prefer rows given this particular ordering of sources.
    source_rows = {}
    for index, row in group.iterrows():
        source = row['SOURCE']
        if source not in source_rows:
            source_rows[source] = []
        source_rows[source].append(row)
    ref_rows_by_source = []
    for source in ['NIFC', 'USFS', 'BLM']:
        if source in source_rows:
            ref_rows_by_source = source_rows[source]
            break
    if len(ref_rows_by_source) == 0:
        raise GroupError("No rows for group by source")

    # Prefer rows with a more specific cause.
    cause_specificity = {1: [], 2: [], 3: []}
    for row in ref_rows_by_source:
        if row['CAUSE_DETAIL_3']:
            cause_specificity[3].append(row)
        elif row['CAUSE_DETAIL_2']:
            cause_specificity[2].append(row)
        else:
            cause_specificity[1].append(row)
    ref_rows_by_pref = []
    for preference in [3, 2, 1]:
        if len(cause_specificity[preference]) > 0:
            ref_rows_by_pref = cause_specificity[preference]
            break
    if len(ref_rows_by_pref) == 0:
        raise GroupError("No rows for group by cause specificity")

    ref_rows = ref_rows_by_pref

    # Identify and report conflicts in ref_rows.
    conflicts = []
        
    agencies = set([e['AGENCY'] for e in ref_rows])
    if len(agencies) > 1: conflicts.append("More than 1 agency: {}".format(agencies))

    cause1 = set([e['CAUSE_DETAIL_1'] for e in ref_rows])
    cause1_f = [e for e in cause1 if e != "undetermined"]
    if len(cause1_f) > 1: conflicts.append("More than one 1st-order cause: {}".format(cause1_f))

    cause2 = set([e['CAUSE_DETAIL_2'] for e in ref_rows])
    if len(cause2) > 1: conflicts.append("More than one 2nd-order cause: {}".format(cause2))

    cause3 = set([e['CAUSE_DETAIL_3'] for e in ref_rows])
    if len(cause3) > 1: conflicts.append("More than one 3rd-order cause: {}".format(cause3))

    start = set([e['STARTDATE'] for e in ref_rows if e['STARTDATE']])
    start_f = [e for e in start]
    if len(start_f) > 1: conflicts.append("More than one startdate: {}".format(start_f))

    end = set([e['ENDDATE'] for e in ref_rows if e['ENDDATE']])
    end_f = set([e for e in end])
    if len(end_f) > 1: conflicts.append("More than one enddate: {}".format(end_f))

    geos = [e['geometry'] for e in ref_rows]
    multiple = False
    for i in range(len(geos)):
        geo1 = geos[i]
        for j in range(i+1, len(geos)):
            geo2 = geos[j]
            if not geo1.equals_exact(geo2, 1e-6):
                multiple = True
                break
            if multiple:
                break
    if multiple:
        conflicts.append("More than one geometry")

    #if len(ref_rows) > 1:
    #    if len(conflicts) > 0:
    #        for conflict in conflicts:
    #            print(conflict)
    #        print()

    # Somewhat arbitrarily choose the last entry.
    # This is deterministic assuming rows maintain order throughout all previous operations.
    final_row = ref_rows[-1]
    return final_row

fires_list = []
for _, group in fire_groups:
    try:
        row = analysis(group)
    except GroupError as e:
        print("ERROR: {}".format(e))
        continue
    fires_list.append(row)

In [20]:
fires_df = gpd.GeoDataFrame(fires_list)
print("Dedupped from {} to {} rows".format(len(all_fires_df.index), len(fires_df.index)))
fires_df.head()

Dedupped from 78188 to 72519 rows


,NAME,YEAR,AGENCY,CAUSE_DETAIL_2,STARTDATE,geometry,ENDDATE,SOURCE,CAUSE_DETAIL_1,CAUSE_DETAIL_3,CENTROID,AREA,COLOR
4829,\tma07038,2022,Private,None,2022-04-28,"POLYGON ((-71.77159 42.26305, -71.77157 42.263...",None,NIFC,None,None,POINT (-71.77133420485544 42.26293914299161),1.612895e+03,0
31743,"""10""",2003,None,lightning,2003-09-03,"POLYGON ((-124.08417 40.13432, -124.08311 40.1...",None,USFS,natural,None,POINT (-124.08479374925035 40.132074317299896),8.615100e+05,0
42612,"""2000""",2000,USFS,incendiary,2000-01-12,"POLYGON ((-86.18311 33.17807, -86.18311 33.178...",None,USFS,human,None,POINT (-86.18298919368247 33.1786008440923),6.623443e+03,0
4030,"""a"" team",1981,NaN,None,1981-08-25,"POLYGON ((-114.51047 42.97429, -114.50870 42.9...",1981-08-25,BLM,human,None,POINT (-114.50622624206164 42.97085724985183),4.391489e+05,0
4131,"""b"" team",1981,NaN,None,1981-09-22,"POLYGON ((-113.82341 43.40338, -113.82336 43.4...",1981-09-24,BLM,human,None,POINT (-113.81399321295667 43.40560198296297),1.804363e+06,0


In [21]:
# Create a df ready to export as a shapefile.

out_df = fires_df.copy()

# export to shapefile does not handle GeometryDtype
out_df = out_df.drop(columns=['CENTROID'])

# set projection so can properly save geometry
out_df.set_crs("EPSG:4326", inplace=True)

# Convert dataframe to shapefile to analyze in QGIS.
out_df.to_file("data/fire_groups.shp", driver='ESRI Shapefile')

/tmp/ipykernel_30808/4095619619.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  out_df.to_file("data/fire_groups.shp", driver='ESRI Shapefile')
